# **LLM Agents** with LangChain and OpenAI

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU
!pip install duckduckgo-search -qU
!pip install langchainhub -qU

In [2]:
import os
from google.colab import userdata

### Initialize OpenAI LLM

In [3]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

### Initialize DuckDuckGo Search Tool

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
search = DuckDuckGoSearchRun()

search

DuckDuckGoSearchRun()

### Load Prompt Template from LangChainHub

In [5]:
from langchain import hub

# Pull the prompt template from LangChainHub
prompt = hub.pull("hwchase17/react")

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


### Create Built-in Agent

In [6]:
from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

In [7]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### Invoke the agent executor with an example question

In [8]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide an accurate answer.
Action: duckduckgo_search
Action Input: machine learning definitionLearn what machine learning is, how it works, and the different types of it powering the services and applications we rely on every day. Explore real-world examples, benefits and dangers, and beginner-friendly courses on Coursera. Machine learning is a form of AI that can adapt to a wide range of inputs, including large data sets and human instruction. Learn how machine learning evolved, how it differs from deep learning and generative AI, and what kinds of neural networks are used in deep learning. Machine learning is a branch of artificial intelligence that equips computer systems to learn and improve from experience without explicit programming. Learn about the history, applications, and potential of this transformative technology in this comprehensive guide for beginners. Machine learning is a 

In [9]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for the word count of an introduction to machine learning.
Action: duckduckgo_search
Action Input: word count of introduction to machine learningAn Introduction to Machine Learning. Machine learning (ML) is a type of artificial intelligence (AI) that allows computers to learn without being explicitly programmed. This article explores the concept of machine learning, providing various definitions and discussing its applications. The article also dives into different classifications of ... Welcome to Introduction to Machine Learning! This course introduces machine learning ( ML) concepts. This course does not cover how to implement ML or work with data. Estimated Course Length: 20 minutes. Learning objectives: Understand the different types of machine learning. Understand the key concepts of supervised machine learning. As a Machine Learning Researcher or Machine Learning Engineer, there are many technical tools and programming lang

### Define Custom Tool

In [10]:
from langchain.agents import tool

@tool
def word_count(text: str) -> int:
  """Returns the word count."""
  return len(text.split())

In [11]:
# Add the custom tool to the list of tools
tools.append(word_count)

print(tools)

[DuckDuckGoSearchRun(), StructuredTool(name='word_count', description='Returns the word count.', args_schema=<class 'pydantic.v1.main.word_countSchema'>, func=<function word_count at 0x7a0793dd5a20>)]


In [12]:
# Create a new agent with the DuckDuckGo search tool and custom word count tool
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

### Invoke the custom agent executor with example questions

In [13]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide an accurate answer.
Action: duckduckgo_search
Action Input: "definition of machine learning"Learn what machine learning is, how it works, and the different types of it powering the services and applications we rely on every day. Explore real-world examples, benefits and dangers, and beginner-friendly courses on Coursera. Machine learning is a broad umbrella term encompassing various algorithms and techniques that enable computer systems to learn and improve from data without explicit programming. It focuses on developing models that can automatically analyze and interpret data, identify patterns, and make predictions or decisions. Machine learning is a form of AI that can adapt to a wide range of inputs, including large data sets and human instruction. Learn how machine learning evolved, how it differs from deep learning and generative AI, and what kinds of neural networks are used i

In [14]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the word_count tool to find the number of words in the text "introduction to machine learning".
Action: word_count
Action Input: "introduction to machine learning"4I need to search for the text "introduction to machine learning" to get the accurate word count.
Action: duckduckgo_search
Action Input: "introduction to machine learning"Learn what machine learning is, how it works, and the different types of it powering the services and applications we rely on every day. Explore real-world examples, benefits and dangers, and beginner-friendly courses on Coursera. Learn the basics of machine learning, a subfield of artificial intelligence that involves the development of algorithms and models that enable computers to improve their performance in tasks through experience. Explore the definitions, types, examples, and applications of machine learning, and how to learn practical skills with Python. Welcome to Introduction to Machine Learning